# 制作局部扫描伪彩图

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#来源于网络资源" data-toc-modified-id="来源于网络资源-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>来源于网络资源</a></span><ul class="toc-item"><li><span><a href="#获取文件" data-toc-modified-id="获取文件-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>获取文件</a></span></li><li><span><a href="#主要功能部分" data-toc-modified-id="主要功能部分-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>主要功能部分</a></span></li><li><span><a href="#主图与聚焦图之间的连接" data-toc-modified-id="主图与聚焦图之间的连接-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>主图与聚焦图之间的连接</a></span></li><li><span><a href="#开始绘图" data-toc-modified-id="开始绘图-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>开始绘图</a></span></li></ul></li><li><span><a href="#实现雷达扫描gif图" data-toc-modified-id="实现雷达扫描gif图-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>实现雷达扫描gif图</a></span></li></ul></div>

## 来源于网络资源
[如何使用Python制作学术动图](https://mp.weixin.qq.com/s/KjHkHYslFnaHOG-ok05Crw)

In [1]:
from matplotlib.transforms import Bbox, TransformedBbox, blended_transform_factory
from mpl_toolkits.axes_grid1.inset_locator import BboxPatch, BboxConnector, BboxConnectorPatch
import matplotlib.pyplot as plt
from math import sin, cos, radians, asin, sqrt
import pandas as pd
import numpy as np
import netCDF4 as nc
import imageio
import contextlib

### 获取文件

In [2]:
file2read = r'../data/otn200_sci_water_temp_live.csv'
ncfile = r'../data/usgsCeSrtm30v1_6ebb_eec1_d277.nc'
data = pd.read_csv(file2read)
data

,unixtime,lat,lon,depth,sci_water_temp
0,1.370530e+09,44.534030,-63.393878,0.083147,0.0000
1,1.370530e+09,44.534115,-63.393787,0.795917,0.0000
2,1.370530e+09,44.534239,-63.393933,4.311246,9.0907
3,1.370531e+09,44.533788,-63.393564,38.666863,0.0000
4,1.370531e+09,44.533654,-63.393454,34.513876,3.3461
...,...,...,...,...,...
15220,1.372088e+09,43.961798,-63.112821,0.078187,12.8826
15221,1.372088e+09,43.961699,-63.112795,0.243324,12.8930
15222,1.372088e+09,43.961690,-63.112795,0.526340,12.8907
15223,1.372088e+09,43.961690,-63.112795,0.707413,12.8890


In [3]:
date = data.unixtime
lat = data.lat
lon = data.lon
depth = data.depth
temp = data.sci_water_temp

In [4]:
etopo2 = nc.Dataset(ncfile)
latDepth = etopo2.variables['latitude'][:]
lonDepth = etopo2.variables['longitude'][:]
topo = etopo2.variables['topo'][:]

In [5]:
def haversine(lon1, lat1, lon2, lat2):  # 经度1，纬度1，经度2，纬度2 （十进制度数）    
    """using haversin to Calculate the great circle distance between 
    two points on the earth (specified in decimal degrees)""" 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])    
    # haversine    
    dlon = lon2 - lon1    
    dlat = lat2 - lat1    
    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    c = 2 * asin(sqrt(a))    
    r = 6371
    return c * r

In [6]:
# Compute distance along transect
dist = np.zeros((np.size(lon)))
for i in range(1, np.size(lon)):
    dist[i] = dist[i - 1] + haversine(lon[i - 1], lat[i - 1], lon[i], lat[i])

In [7]:
bathy = np.zeros((np.size(lon)))
for i in range(np.size(lon)):
    cost_func = np.abs(lonDepth - lon[i])
    xmin = np.where(cost_func == np.min(cost_func))[0]
    cost_func = np.abs(latDepth - lat[i])
    ymin = np.where(cost_func == np.min(cost_func))[0]
    bathy[i] = -topo[ymin, xmin]

### 主要功能部分
实现局部放大功能：
依据ax1:主要绘图部分，ax2定位聚焦部分，xmin和xmax设置聚焦x轴向的范围。
+ 考虑到绘制伪彩图，优化了绘图部分使用上下文管理释放连接，可以减少绘图所耗的资源！

In [8]:
@contextlib.contextmanager
def zoom_effect(ax1, ax2, xmin, xmax, **kwargs):
    """
    ax1 : the main axes
    ax1 : the zoomed axes
    (xmin,xmax) : the limits of the colored area in both plot axes.
    connect ax1 & ax2. The x-range of (xmin, xmax) in both axes will
    be marked. The keywords parameters will be used ti create
    patches.
    Source: http://matplotlib.org/dev/users/annotations_guide.html#zoom-effect-between-axes
    """
    trans1 = blended_transform_factory(ax1.transData, ax1.transAxes)
    trans2 = blended_transform_factory(ax2.transData, ax2.transAxes)
    bbox = Bbox.from_extents(xmin, 0, xmax, 1)
    mybbox1 = TransformedBbox(bbox, trans1)
    mybbox2 = TransformedBbox(bbox, trans2)
    prop_patches = kwargs.copy()
    prop_patches["ec"] = "r"
    prop_patches["alpha"] = None
    prop_patches["facecolor"] = 'none'
    prop_patches["linewidth"] = 2
    c1, c2, bbox_patch1, bbox_patch2, p = \
            connect_bbox(mybbox1, mybbox2, loc1a=3, loc2a=2, 
                         loc1b=4, loc2b=1, prop_lines=kwargs, prop_patches=prop_patches)
    ax1.add_patch(bbox_patch1)
    ax2.add_patch(bbox_patch2)
    ax2.add_patch(c1)
    ax2.add_patch(c2)
    ax2.add_patch(p)
    yield c1, c2, bbox_patch1, bbox_patch2, p
    bbox_patch1.remove()
    bbox_patch2.remove()
    c1.remove()
    c2.remove()
    p.remove()

### 主图与聚焦图之间的连接
由zoom调用，可以自己设置连接部分

In [9]:
def connect_bbox(bbox1, bbox2, loc1a, loc2a, loc1b, loc2b, prop_lines, prop_patches=None):
    # Fuctions for zoom effect ****************************************************
    # Source: http://matplotlib.org/dev/users/annotations_guide.html#zoom-effect-between-axes
    if prop_patches is None:
        prop_patches = prop_lines.copy()
        prop_patches["alpha"] = prop_patches.get("alpha", 1) * 0.2
    c1 = BboxConnector(bbox1, bbox2, loc1=loc1a, loc2=loc2a, **prop_lines)
    c1.set_clip_on(False)
    c2 = BboxConnector(bbox1, bbox2, loc1=loc1b, loc2=loc2b, **prop_lines)
    c2.set_clip_on(False)
    bbox_patch1 = BboxPatch(bbox1, **prop_patches)
    bbox_patch2 = BboxPatch(bbox2, **prop_patches)
    p = BboxConnectorPatch(bbox1, bbox2,
    # loc1a=3, loc2a=2, loc1b=4, loc2b=1,
    loc1a=loc1a, loc2a=loc2a, loc1b=loc1b, loc2b=loc2b, **prop_patches)
    p.set_clip_on(False)
    return c1, c2, bbox_patch1, bbox_patch2, p

### 开始绘图

In [10]:
# Make plots
nframes = 20
overlap = 0.95
window = np.floor(max(dist) - min(dist)) / (nframes - (nframes * overlap) + overlap)
xmin = 0
xmax = xmin + window
cmp = plt.cm.get_cmap('jet', 16)

ax1 = plt.subplot(211)
plt.fill_between(dist, bathy, 1000, color='k')
plt.scatter(dist, depth, s=15, c=temp,
            cmap = cmp, marker='o', edgecolor='none')
plt.ylim((-0.5, max(depth) + 5))
ax1.set_ylim(ax1.get_ylim()[::-1])
cbar = plt.colorbar(orientation='vertical', extend='both')
cbar.ax.set_ylabel('Temperature ($^\circ$C)')
plt.title('OTN Glider transect')
plt.ylabel('Depth (m)')
ax1.set_xlim(min(dist), max(dist))
ax2 = plt.subplot(212)
plt.fill_between(dist, bathy, 1000, color='k')
plt.scatter(dist, depth, s=15, c=temp,
            cmap= cmp, marker='o', edgecolor='none')
plt.ylim((-0.5, max(depth) + 5))
ax2.set_ylim(ax2.get_ylim()[::-1])
plt.ylabel('Depth (m)')
plt.xlabel('Distance along transect (km)')
ax2.set_xlim(xmin, xmax)
for i in range(0, nframes):
    ax2.set_xlim(xmin, xmax)
    with zoom_effect(ax1, ax2, xmin, xmax) as zoom:
        file2write = 'glider_' + '%02d'%i + '.png'
        plt.savefig(file2write, bbox_inches='tight')
    xmin = xmax - np.floor(window * overlap)
    xmax = xmin + window
plt.close()

In [11]:
imagelist = ['glider_'+'%02d'%i +'.png' for i in range(0,20)]
frames = [imageio.imread(_) for _ in imagelist]
outname = 'gliders.gif'
imageio.mimsave(outname, frames, 'GIF', duration=0.3)

![gif](https://i.loli.net/2020/08/20/ueTD4gUih8dX5WV.gif)

![](/Users/caizhiming/Documents/opensource/MyRepos/DataScience/data/pic/gliders.gif)

## 实现雷达扫描gif图

In [10]:
file = r'../data/lidarData.txt'
df = pd.read_csv(file, index_col=0)
# 以列为单位丢弃含有nan值的列
df.dropna(axis=1, inplace=True)
df

,2019/09/19 00:00:30,2019/09/19 00:05:23,2019/09/19 00:10:16,2019/09/19 00:15:09,2019/09/19 00:20:03,2019/09/19 00:24:56,2019/09/19 00:29:49,2019/09/19 00:34:43,2019/09/19 00:39:36,2019/09/19 00:44:29,...,2019/09/19 23:08:53,2019/09/19 23:13:47,2019/09/19 23:18:40,2019/09/19 23:23:33,2019/09/19 23:28:27,2019/09/19 23:33:20,2019/09/19 23:38:13,2019/09/19 23:43:07,2019/09/19 23:48:00,2019/09/19 23:52:53
Height(km)\Time,,,,,,,,,,,,,,,,,,,,,
0.0000,0.006,0.000,0.005,0.000,0.000,0.003,0.006,0.003,0.001,0.002,...,0.004,0.000,0.002,0.001,0.005,0.001,0.001,0.000,0.001,0.000
0.0075,0.009,0.000,0.008,0.000,0.000,0.005,0.010,0.005,0.001,0.004,...,0.006,0.000,0.004,0.002,0.008,0.001,0.002,0.001,0.002,0.000
0.0150,0.011,0.000,0.009,0.000,0.000,0.006,0.011,0.006,0.001,0.005,...,0.007,0.000,0.004,0.002,0.010,0.002,0.002,0.001,0.003,0.000
0.0225,0.012,0.000,0.010,0.000,0.000,0.006,0.012,0.006,0.002,0.005,...,0.007,0.000,0.005,0.002,0.010,0.002,0.002,0.001,0.003,0.000
0.0300,0.012,0.000,0.010,0.000,0.000,0.007,0.013,0.006,0.002,0.005,...,0.007,0.000,0.005,0.002,0.011,0.002,0.002,0.001,0.003,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14.9700,0.019,0.019,0.017,0.020,0.016,0.013,0.018,0.020,0.014,0.018,...,0.004,0.005,0.008,0.005,0.003,0.005,0.005,0.002,0.005,0.004
14.9775,0.009,0.009,0.008,0.010,0.008,0.005,0.009,0.010,0.006,0.009,...,0.000,0.001,0.002,0.001,0.002,0.001,0.000,0.001,0.000,0.000
14.9850,0.001,0.002,0.001,0.002,0.001,0.003,0.001,0.002,0.000,0.001,...,0.000,0.001,0.001,0.001,0.001,0.001,0.000,0.001,0.000,0.000


In [11]:
def lidar_pic(data, axes, show_height=5, hide=True):
    # 获取数据长度及对应雷达数据最大高度
    H, _height = len(data), data.index[len(data)-1]
    # 依据显示高度截取数据
    _sli = int(H/_height*show_height)
    height=data.iloc[:,0]
    
    tic = H // _height
    if hide:
        # 处理盲区
        _data = data.iloc[int(H/_height*0.2):_sli, :]
        s_tic = np.arange(1., 1.+show_height)
        d_tic = (s_tic - 0.2) * tic
        d_tic = [0] + list(d_tic)
        s_tic = [0.2] + list(s_tic)
    else:
        _data = data.iloc[:_sli, :]
        s_tic = np.arange(.0, 1.+show_height)
        d_tic = s_tic * tic

    
    p=axes.imshow(_data, vmax=1,vmin=0,origin="lower",aspect='auto', cmap='jet')

    idx = list(axes.get_xticks()[1:].astype(int))[:-1]

    axes.set_yticks(d_tic)
    axes.set_yticklabels(s_tic)
    axes.set_xticks(idx)
    axes.set_xticklabels([data.columns[item][:-3].replace(' ', '\n') for item in idx])
    return p

In [12]:
%%time
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10))
ax1 = plt.subplot(211)
ax2 = plt.subplot(212)
p = lidar_pic(df, ax1)
lidar_pic(df, ax2)
ax1.set_ylabel('Height/Km')
fig.colorbar(p, ax=ax1)

nframes = 20
overlap = 0.95
window = (df.shape[-1]-1) / (nframes*(1-overlap) + overlap)
xmin = 0
xmax = xmin + window

ax2.set_xlim(xmin, xmax)
for i in range(0, nframes):
    ax2.set_xlim(xmin, xmax)
    with zoom_effect(ax1, ax2, xmin, xmax) as zoom:
        file2write = '../data/pic/lidar_gif/' + 'lidar_' + '%02d'%i + '.png'
        plt.savefig(file2write)
    xmin = xmin + (1-overlap)*window
    xmax = xmin + window
plt.close()

CPU times: user 7.96 s, sys: 656 ms, total: 8.61 s
Wall time: 3.98 s


In [13]:
imagelist = ['../data/pic/lidar_gif/'+'lidar_'+'%02d'%i +'.png' for i in range(0,20)]
frames = [imageio.imread(_) for _ in imagelist]
outname = '../data/pic/lidar_gif/lidar_.gif'
imageio.mimsave(outname, frames, 'GIF', duration=0.6)

![](https://i.loli.net/2020/08/20/SJlWRsnHtOaEeIA.gif)